In [2]:
import tensorflow as tf 
import keras as kr 
import numpy as np 

In [5]:
fashion_data = kr.datasets.fashion_mnist

fashion_data

<module 'keras.api.datasets.fashion_mnist' from 'c:\\Users\\jayku\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\keras\\api\\datasets\\fashion_mnist\\__init__.py'>

In [6]:
(train_image, train_labels), (test_image, test_labels) = fashion_data.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 3us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 91s 3us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 3us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 10s 2us/step


In [7]:
test_image = test_image/255.0
train_image = train_image/255.0

In [8]:
train_image = train_image.reshape(len(train_image), 28,28,1)
test_image = test_image.reshape(len(test_image), 28,28,1)


In [20]:
def build_model(hp):
    
    model = kr.Sequential([
        kr.layers.Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
            activation='relu',
            input_shape=(28,28,1)
        ),
        kr.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    kr.layers.Flatten(),
    kr.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
     kr.layers.Dense(10, activation='softmax')
    ])
      
    model.compile(optimizer=kr.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    
    return model

In [21]:
from kerastuner import RandomSearch 
from kerastuner.engine.hyperparameters import HyperParameters

In [26]:
tuner_search=RandomSearch(build_model, objective='val_accuracy',
                          max_trials=5, directory='output', project_name='Mnist Fashion Dataset')

In [27]:
tuner_search.search(train_image, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 02m 09s]
val_accuracy: 0.9003333449363708

Best val_accuracy So Far: 0.903333306312561
Total elapsed time: 00h 15m 43s


In [28]:
model=tuner_search.get_best_models(num_models=1)[0]

c:\Users\jayku\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [29]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 24, 24, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 20, 20, 48)     │        76,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 19200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 80)             │     1,536,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,615,402 (6.16 MB)

 Trainable params: 1,615,402 (6.16 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.fit(train_image, train_labels, epochs=4, validation_split=0.1, initial_epoch=3)

Epoch 4/4
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 63s 37ms/step - accuracy: 0.9177 - loss: 0.2211 - val_accuracy: 0.9092 - val_loss: 0.2481
